In [1]:
import math
import rasterio as rio

import numpy as np

from sklearn.preprocessing import scale
from sklearn.decomposition import PCA as sklearnPCA

# For the mathematical side of PCA
import operator

### Importing the Image

In [2]:
image_file = r"C:\Users\RQ\Documents\PCA\GC_Landsat8_SR.tif"
sat_data = rio.open(image_file)

### Calculating the dimensions of the image on Earth in metres

In [3]:
width_in_projected_units = sat_data.bounds.right - sat_data.bounds.left
height_in_projected_units = sat_data.bounds.top - sat_data.bounds.bottom

print('Width: {}, Height: {}'.format(width_in_projected_units, height_in_projected_units))

Width: 0.08556453081239113, Height: 0.1691078522355003


### Convert pixel co-ordinates to longtidues and latitudes


In [4]:
# Upper left pixel
row_min = 0
col_min = 0

# Lower right pizel. Rows and colums are zero indexing.
row_max = sat_data.height - 1 
col_max = sat_data.width - 1 

# Transform coordinates with the dataset's affine transformation
topleft = sat_data.transform * (row_min, col_min)
botright = sat_data.transform * (row_max, col_max)

print("Top left corner coordinates: {}".format(topleft))
print("Bottom right corner coordinates: {}".format(botright))

Top left corner coordinates: (-74.7413587747272, 7.166669505177131)
Bottom right corner coordinates: (-74.57238566978432, 7.081239721657364)


### Bands
Let's check how many bands are in our image.

In [5]:
print('Bands: {}'.format(sat_data.count))

# Sequence of band indexes
print(sat_data.indexes)

Bands: 12
(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12)


### Data Preprocessing

In [6]:
src_meta = sat_data.meta

src_meta

{'driver': 'GTiff',
 'dtype': 'float64',
 'nodata': None,
 'width': 635,
 'height': 1255,
 'count': 12,
 'crs': CRS.from_dict(init='epsg:4326'),
 'transform': Affine(0.00013474729261792824, 0.0, -74.7413587747272,
        0.0, -0.00013474729261792824, 7.166669505177131)}

In [19]:
band_lst = None

In [20]:
img

array([[[  nan,   nan,   nan, ...,   nan,   nan,   nan],
        [310. , 310. , 235. , ..., 229. , 274.5, 274.5],
        [310. , 310. , 235. , ..., 243. , 285. , 285. ],
        ...,
        [230. , 237. , 237. , ..., 148. , 148. , 153.5],
        [230. , 237. , 237. , ..., 148. , 148. , 153.5],
        [230. , 306. , 306. , ..., 143.5, 143.5, 167.5]],

       [[  nan,   nan,   nan, ...,   nan,   nan,   nan],
        [360. , 360. , 245. , ..., 255.5, 298. , 298. ],
        [360. , 360. , 245. , ..., 264. , 296. , 296. ],
        ...,
        [314. , 308. , 308. , ..., 215.5, 215.5, 232.5],
        [314. , 308. , 308. , ..., 215.5, 215.5, 232.5],
        [310. , 396. , 396. , ..., 236.5, 236.5, 284. ]],

       [[  nan,   nan,   nan, ...,   nan,   nan,   nan],
        [652. , 652. , 526. , ..., 457. , 558.5, 558.5],
        [652. , 652. , 526. , ..., 467.5, 553. , 553. ],
        ...,
        [712. , 706.5, 706.5, ..., 595. , 595. , 628.5],
        [712. , 706.5, 706.5, ..., 595. , 595

In [21]:
src_meta = sat_data.meta
n_bands = src_meta['count']
height = src_meta['height']
width = src_meta['width']

n_bands, height, width

(12, 1255, 635)

In [22]:
# Read the image
if band_lst!=None:
     img = sat_data.read(band_lst)
     n_bands = len(band_lst)
else:
    img = sat_data.read()

In [23]:
img.shape

(12, 1255, 635)

In [24]:
n_bands

12

In [10]:
n_pixel = height*width

n_pixel

796925

In [11]:
# Goal: We want to convert (n_bands, height,width) = (12, 1255, 635) into an array that is (n_pixel, n_bands) = (796925, 12)
flattened_img = img.reshape(n_bands, -1)
flattened_img = flattened_img.T 
flattened_img.shape

(796925, 12)

In [12]:
flattened_img

array([[  nan,   nan,   nan, ...,   nan,   nan,   nan],
       [  nan,   nan,   nan, ...,   nan,   nan,   nan],
       [  nan,   nan,   nan, ...,   nan,   nan,   nan],
       ...,
       [143.5, 236.5, 693. , ..., 130. , 322. ,   0. ],
       [143.5, 236.5, 693. , ..., 130. , 322. ,   0. ],
       [167.5, 284. , 793.5, ..., 160. , 322. ,   0. ]])

### Standardizing the Data
We need to remove the null data from the image and then standardize the data so we can use the non-nulls for the PCA.

In [13]:
# Separte the null and not null arrays
notnull_array = flattened_img[~np.isnan(flattened_img).all(axis=1)]

In [14]:
# Create an empty array for the results
PCA_results = np.empty(flattened_img.shape)

PCA_results

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [15]:
# First we standardize the not null data for the PCA 
std_array = scale(notnull_array, axis=0)

std_array.shape

(796290, 12)

In [16]:
# Let's find out how many rows of NaNs we dropped
flattened_img.shape[0] - std_array.shape[0]

635

### Principal Component Analysis

In [17]:
# Principal Component Analysis for the standardized_img
sklearn_pca = sklearnPCA(n_components=n_bands)
result = sklearn_pca.fit_transform(std_array)

In [18]:
# Shape of results
result.shape

(796290, 12)

## Creating the New Image Array

We need to replace the correct rows of PCA_results with the NaNs and with the appropriate PCA results.

In [19]:
# Get the indices of the nulls 
null_idx = np.isnan(flattened_img).any(axis=1)

# Give PCA results the same nulls as flattened_img (same rows based on index)
PCA_results[null_idx] = flattened_img[null_idx]

PCA_results

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [20]:
# Get the row index for all not nulls
notnull_idx = ~np.isnan(flattened_img).any(axis=1)

PCA_results[notnull_idx]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [21]:
PCA_results[notnull_idx].shape, result[~np.isnan(result).any(axis=1)].shape

((796290, 12), (796290, 12))

We can see that the result and PCA_results have the same shape, so we need to change the array so the PCA_results is correct.

In [22]:
# Change the PCA_results to include the not null pca component of the image
PCA_results[notnull_idx] = result[~np.isnan(result).any(axis=1)]

# Let's view our changed data now 
PCA_results

array([[        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       ...,
       [-1.11915996,  1.12836753, -0.26457019, ..., -0.0530736 ,
         0.01543224,  0.        ],
       [-1.11915996,  1.12836753, -0.26457019, ..., -0.0530736 ,
         0.01543224,  0.        ],
       [-0.51470015,  1.64714683,  1.4621112 , ..., -0.03583775,
        -0.00821718,  0.        ]])

### Reshaping Data back into Image

We first need to convert this DataFrame back into an array, and then get it back into the shape of (n_bands, height, width) so that we can export it as an image with the same metadata as the original file.

#### Resulting PCA Image Array

In [23]:
PCA_results.shape

(796925, 12)

In [24]:
# Transpose the data back since we originally had flattened_img be (12, 796925)
PCA_results = PCA_results.T

PCA_results.shape

(12, 796925)

In [25]:
# We need to reshape this to be (n_bands, height, width)
new_img = PCA_results.reshape(n_bands, height, width)

new_img.shape

(12, 1255, 635)

In [26]:
# Let's take a look at our new image data
new_img

array([[[            nan,             nan,             nan, ...,
                     nan,             nan,             nan],
        [ 1.55981573e+00,  1.55981573e+00, -5.10354611e-01, ...,
         -1.73492470e+00, -1.75189884e-01, -1.75189884e-01],
        [ 1.55981573e+00,  1.55981573e+00, -5.10354611e-01, ...,
         -1.69247913e+00, -2.20450097e-01, -2.20450097e-01],
        ...,
        [ 1.77722615e+00,  1.63097247e+00,  1.63097247e+00, ...,
         -1.53239811e+00, -1.53239811e+00, -1.27035894e+00],
        [ 1.77722615e+00,  1.63097247e+00,  1.63097247e+00, ...,
         -1.53239811e+00, -1.53239811e+00, -1.27035894e+00],
        [ 1.78990240e+00,  3.45441678e+00,  3.45441678e+00, ...,
         -1.11915996e+00, -1.11915996e+00, -5.14700148e-01]],

       [[            nan,             nan,             nan, ...,
                     nan,             nan,             nan],
        [-1.15308829e+00, -1.15308829e+00, -1.69287019e+00, ...,
         -2.96035486e-01, -1.58171825e

In [27]:
# Make a copy of the source dictionary 
dst_meta = src_meta.copy()

# Open a new file in 'write' mode and unpack (**) the destination metadata
dst_fp = r"C:\Users\RQ\Documents\PCA\GC_Landsat8_Output.tif"
with rio.open(dst_fp, 'w', **dst_meta) as dst:
    dst.write(new_img)

This next section is just the mathematical steps done individually to understand how Principal Component Analysis works!

# Mathematical Steps Behind PCA

## 1) Eigendecomposition
### Computing Eigenvectors and Eigenvalues
We willl begin by calculating the eigenvectors (principal components) to determine the direction of the new feature space and the eigenvalues to determine their magnitude.

### Covariance Matrix
Classic approach to perform the eigendecomposition on the covariance matrix (dxd matrix where each element is the covariance between two features).

In [29]:
cov_mat = np.cov(std_array.T)

print('Covariance matrix \n%s' %cov_mat)

Covariance matrix 
[[ 1.00000126  0.96389843  0.84098934  0.857109   -0.07672584  0.68493906
   0.79172728  0.42243999  0.34386798  0.10049293  0.03269933  0.        ]
 [ 0.96389843  1.00000126  0.92090487  0.93293626 -0.11077775  0.74554755
   0.86929467  0.46377328  0.37324319  0.2033055   0.03231332  0.        ]
 [ 0.84098934  0.92090487  1.00000126  0.95288191  0.01710439  0.84159462
   0.90518254  0.46506024  0.36422715  0.26377601  0.02288368  0.        ]
 [ 0.857109    0.93293626  0.95288191  1.00000126 -0.1686747   0.80179966
   0.91092935  0.49055129  0.39730182  0.24612058  0.01903937  0.        ]
 [-0.07672584 -0.11077775  0.01710439 -0.1686747   1.00000126  0.2361911
  -0.03044682 -0.06555473 -0.06178892 -0.12807403 -0.01273185  0.        ]
 [ 0.68493906  0.74554755  0.84159462  0.80179966  0.2361911   1.00000126
   0.92681582  0.50918554  0.4160753   0.13640404  0.00363407  0.        ]
 [ 0.79172728  0.86929467  0.90518254  0.91092935 -0.03044682  0.92681582
   1.00000126 

In [30]:
# Eigen decomposition of covariance matrix
eig_vals, eig_vecs = np.linalg.eig(cov_mat)

# View the eigenvalues and eigenvectors
print('Eigenvalues \n%s' %eig_vals)
print('\nEigenvectors \n%s' %eig_vecs)

Eigenvalues 
[5.95161719 1.45994458 1.17749041 0.9998787  0.86347025 0.32016425
 0.11641017 0.01270501 0.04303238 0.02867525 0.02662563 0.        ]

Eigenvectors 
[[-3.62138710e-01 -1.42443825e-01 -7.96804368e-03 -3.77633055e-02
   2.77526150e-01 -5.46150335e-01 -4.30310979e-01 -4.63650547e-01
  -3.73689347e-02 -2.57395098e-01  6.31407880e-02  0.00000000e+00]
 [-3.85940997e-01 -1.60575273e-01  3.84856588e-02 -1.73122241e-02
   1.73628137e-01 -3.42113288e-01 -7.08406689e-02  7.73823695e-01
   5.54407948e-02  2.63428024e-01  2.55349665e-03  0.00000000e+00]
 [-3.88151689e-01 -1.69906249e-01 -6.84679828e-02  9.04650367e-03
   1.57361207e-03  1.22023860e-02  6.16919362e-01 -2.02067678e-01
   1.07964341e-01  8.39856696e-02  6.12713896e-01  0.00000000e+00]
 [-3.90697379e-01 -1.45749521e-01  8.31924875e-02  1.01872860e-02
   9.97117165e-02  9.42236194e-02  4.58630822e-01 -6.23076883e-02
  -1.81960767e-01 -2.64685325e-01 -6.94891953e-01  0.00000000e+00]
 [ 2.03183134e-02  2.89244541e-02 -8.5419

## 2) Selecting Principal Components

In [33]:
# We make a list of (eigenvalue, eigenvector) tuples or in other words a list of eigenpairs
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

# Sort the eigenpairs from high to low 
eig_pairs.sort(key = operator.itemgetter(0), reverse=True)

print('Eigenvalues in descending order:')
for i in eig_pairs:
    print(i[0])


Eigenvalues in descending order:
5.951617194902354
1.4599445813710743
1.1774904060045845
0.9998786991746794
0.8634702455868085
0.3201642506753802
0.1164101693988662
0.04303237724301667
0.028675250402921255
0.02662563001676745
0.012705009311571258
0.0


In [34]:
lst = []
for i in eig_pairs:
    lst.append(i[0])

lst

[5.951617194902354,
 1.4599445813710743,
 1.1774904060045845,
 0.9998786991746794,
 0.8634702455868085,
 0.3201642506753802,
 0.1164101693988662,
 0.04303237724301667,
 0.028675250402921255,
 0.02662563001676745,
 0.012705009311571258,
 0.0]

In [35]:
# Take all 12 eigevectors with the highest eigenvalues to construct our matrix W
matrix_w = np.hstack(lst)

print('Matrix W:\n', matrix_w)

Matrix W:
 [5.95161719 1.45994458 1.17749041 0.9998787  0.86347025 0.32016425
 0.11641017 0.04303238 0.02867525 0.02662563 0.01270501 0.        ]


In [36]:
matrix_w.shape

(12,)

## 3) Projection Onto the New Feature Space
We use the 635 x 2-dimensional projection matrix W to transform our samples onto the newsubspace via Y = X x W

In [38]:
Y = std_arrayP.dot(matrix_w)

Y

array([ 6.01395204,  6.01395204, -3.17147738, ..., -6.74303572,
       -6.74303572, -4.5014112 ])